In [53]:
# Estrae la lista de TABLAS y VISTAS de un propietario especifico
 
import pyodbc   
 
def getSysTables(propietario, tipoObjeto):

    lista_tablas = []
    propietario_id = ""
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()
    
    sql_usuarios  = "select * from sysusers where name=" + "'" +propietario+ "'"
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        propietario_id=row.suid

    sql_tablas = "select * from systable where table_type='" + tipoObjeto + "' and creator=" + str(propietario_id) + " order by table_name"
    cursor.execute(sql_tablas)
    rows = cursor.fetchall()
    for row in rows:
        tbl={}
        tbl["table_creator"] = propietario
        tbl["table_id"] = row.table_id
        tbl["table_count"]= row.count
        tbl["table_name"]= row.table_name
        lista_tablas.append(tbl)

    cursor.close()
    conexionDB.close()

    return lista_tablas

In [54]:
lista_tablas = getSysTables('PRG', 'BASE')
lista_vistas = getSysTables('PRG', 'VIEW')

file = open("Tablas.txt", "w")
file.write('\n'.join(str(lt) for lt in lista_tablas))
file.close()

file = open("Vistas.txt", "w")
file.write(('\n'.join(str(lt) for lt in lista_vistas)))
file.close()

In [55]:
# Extrae la lista de columnas de la tabla indicada en los parametros

def getSysColumns(propietario, tabla):
    columnas = []
 
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()

    sql_usuarios  = "select * from syscolumns where tname=" + "'" + tabla + "'" + " order by colno"
    
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        obj = {}
        obj["col_name"]=row.cname
        obj["col_type"]=row.coltype
        obj["col_null"]=row.nulls
        obj["col_length"]=row.length
        obj["col_inPK"]=row.in_primary_key
        columnas.append(obj)

    cursor.close()
    conexionDB.close()

    return columnas

In [56]:
# Genera la sentencia DDL de CREATE TABLE de la tabla indicada en los parametros

def createTable(propietario,tabla,columnas):
    sentenciaDDL = ""
    
    sentenciaDDL = "CREATE TABLE " + "\"" + propietario + "\".\"" + tabla +"\"\n" 
    sentenciaDDL = sentenciaDDL + "(" +"\n" 
    for col in columnas:
        sentenciaDDL = sentenciaDDL + "\t" + col["col_name"] 
        if col["col_length"] != "": 
            if col["col_type"] != "long varchar":
                sentenciaDDL = sentenciaDDL + " " + col["col_type"] + "(" + str(col["col_length"]) + ")" 
            else:
                sentenciaDDL = sentenciaDDL + " " + col["col_type"]
        if col["col_null"] == "N":
            sentenciaDDL = sentenciaDDL + " NOT NULL" +",\n"
        else:
            sentenciaDDL = sentenciaDDL + ",\n"

    sentenciaDDL =  sentenciaDDL+ ")" + ";\n\n"

    return sentenciaDDL

In [57]:
# Escribe sentencias DDL de CREATE TABLE para todas las tablas en el archivo PyUnload.sql

todas_columnas =[]

file = open("PyUnload.sql", "a")
for col in lista_tablas:
    columnas = getSysColumns(col["table_creator"], col["table_name"])
    sentenciaDDL = createTable(col["table_creator"], col["table_name"], columnas)
    file.write(sentenciaDDL)

    todas_columnas.append("\n" + col["table_name"])
    todas_columnas.append('\n'.join(str(x) for x in columnas))
file.close()

file = open("Columnas.txt", "w")
file.write(('\n'.join(str(x) for x in todas_columnas)))
file.close()


In [ ]:
# Extrae la lista de indices de la tabla indicada en los parametros

def getSysIndexes(tabla):
    indices = []
 
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()

    sql_indices  = "select * from sysindexes where indextype = 'Non-unique' and tname =" + "'" + table_name + "'"   

    cursor.execute(sql_indices)
    rows = cursor.fetchall()
    for row in rows:
        obj = {}
        obj["ind_name"]=row.iname
        colnames=row.colnames
        obj["ind_columns"] = colnames.split(',')
        indices.append(obj)

    cursor.close()
    conexionDB.close()

    return columnas

In [ ]:
# Genera sentencia DDL de CREATE INDEX para la tabla indicada 

def createIndexes(propietario,table_name):
    # indextype = Non-unique
    # colnames = PLUProducto ASC,Obsoleto ASC

    #CREATE INDEX "Inventario001" ON "PRG"."Inventario"
    #(
	#   "PLUProducto" ASC,
	#   "Fecha" ASC
    #) WITH HASH SIZE 10
    #go
  
    columnas_indice = []
    sentenciaDDL = ""
    
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()

    sql_indices  = "select * from sysindexes where indextype = 'Non-unique' and tname =" + "'" + table_name + "'"   
    
    cursor.execute(sql_indices)
    rows = cursor.fetchall()
  
    for row in rows:
        colnames = row.colnames
        indice_name = row.iname
        columnas_indice = colnames.split(',')

        sentenciaDDL = "CREATE INDEX " + "\"" + indice_name + "\" ON " + "\"" + propietario + "\".\"" + table_name +"\"\n" 
        sentenciaDDL = sentenciaDDL + "(" +"\n" 
        for col in columnas_indice:
            sentenciaDDL = sentenciaDDL + "\t"  + col + ",\n"
        sentenciaDDL = sentenciaDDL + ");" +"\n"

    cursor.close()
    conexionDB.close()

    return sentenciaDDL

In [60]:
# Genera sentencia DDL de CREATE INDEX para la tabla indicada 

def createIndexes(propietario,table_name):
    # indextype = Non-unique
    # colnames = PLUProducto ASC,Obsoleto ASC

    #CREATE INDEX "Inventario001" ON "PRG"."Inventario"
    #(
	#   "PLUProducto" ASC,
	#   "Fecha" ASC
    #) WITH HASH SIZE 10
    #go
  
    columnas_indice = []
    sentenciaDDL = ""
    
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()

    sql_indices  = "select * from sysindexes where indextype = 'Non-unique' and tname =" + "'" + table_name + "'"   
    
    cursor.execute(sql_indices)
    rows = cursor.fetchall()
  
    for row in rows:
        colnames = row.colnames
        indice_name = row.iname
        columnas_indice = colnames.split(',')

        sentenciaDDL = "CREATE INDEX " + "\"" + indice_name + "\" ON " + "\"" + propietario + "\".\"" + table_name +"\"\n" 
        sentenciaDDL = sentenciaDDL + "(" +"\n" 
        for col in columnas_indice:
            sentenciaDDL = sentenciaDDL + "\t"  + col + ",\n"
        sentenciaDDL = sentenciaDDL + ");" +"\n"

    cursor.close()
    conexionDB.close()

    return sentenciaDDL

In [61]:
# Escribe sentencia DDL de CREATE INDEX para todas las tablas en el archivo PyUnload.sql

todos_indices = []
file = open("PyUnload.sql", "a")
for col in lista_tablas:
    sentenciaDDL = createIndexes(col["table_creator"], col["table_name"])
    file.write(sentenciaDDL)

file.close()

In [ ]:
def exportData(table_name, table_id):

    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()
    
    sql  = "select * from " + table_name
    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  str(table_id) + ".dat"
    
    if len(rows)>0:    
        file = open(archivo_dat, "w")
        for row in rows:
            r = str(row)
            r = r.replace('(','')
            r = r.replace(')','')
            r = r.replace('Decimal','')
            file.write(r+"\n")
        file.close()
    else:
        return archivo_dat, False
    
    return archivo_dat , True

In [ ]:
# Exporta los datos de las primaras 7 tablas que tienen registros

archivos_dat = []
n = 1
for col in lista_tablas:
    exportedFile, exportResult = exportData(col["table_name"], col["table_id"])
    if exportResult:
        print (exportedFile)
        archivos_dat.append(col)
        n = n + 1
    if n==8:
        break

In [ ]:
def insertData(dat, archivo_sql):

    print(dat)
   
    archivo_dat = str(dat["table_id"]) + '.dat'

    #print(archivo_dat)
    file = open(archivo_sql, "a")

    sentenciaInsert = "INPUT INTO " + "\"" + dat["table_creator"] + "\".\"" + dat["table_name"] +"\"\n"
    sentenciaInsert = sentenciaInsert + "FROM " +  "\"" + archivo_dat + "\"\n"  
    sentenciaInsert = sentenciaInsert +  "FORMAT ASCII\n"
    sentenciaInsert = sentenciaInsert +  "BY ORDER;\n\n"
    
    file.write(sentenciaInsert)
    file.close()

    return True 

In [ ]:
for dat in archivos_dat:
    insertData(dat, "PyUnload.sql")